In [1]:
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
## Path of file
# file which want to give label [train data/test data]
filename_train = "../data/clean/data.txt"
filename_test = "../data/testing/clear-sent-comb.txt"
filename_vocab = "../data/clean/dict_order.xlsx"
filename_replace_word = "../data/clean/replace quantity.xlsx"

In [3]:
## Create list of vocabulary, list of vocabulary+label
df_vocab = pd.read_excel(filename_vocab)

list_vocab = []
list_vocab_label = []

for col in df_vocab.columns:
    for val in df_vocab[col].dropna():
        list_vocab.append(val.lower().rstrip())
        list_vocab_label.append(val.lower().rstrip()+':'+col)

In [4]:
## Load train/test data
data = []
with open(filename_test, encoding='utf-8') as f:
    for i in f:
        data.append(i.strip().lower())

In [5]:
## Replace 'Sesachet' -> 'satu sachet'
df_replace_word = pd.read_excel(filename_replace_word)
dict_replace_word = pd.Series(df_replace_word.after.values, index=df_replace_word.before).to_dict()

new_data = []

for line in tqdm(data):
    check_status = 0
    for key, val in dict_replace_word.items():
        if key in line:
            check_status = 1
            new_data.append(line.replace(key,val))
    if check_status != 1:
        new_data.append(line)

In [6]:
'''
    Give label based on special conditions or even without conditions
'''
result = []
out_of_vocab = []

for i in tqdm(new_data):
    list_label = []
    tokenize_word = i.split()
    for token in tokenize_word :
        if token in list_vocab:
            if ((token == 'kopi') or (token == 'susu')) and (tokenize_word.index(token) == 0):
                list_label.append('item')
            elif (token == 'delapan') and ('tujuh' not in i):
                list_label.append('numeric')
            elif (token == 'cap') and (('lang' in i) or ('badak' in i)):
                list_label.append('b-brand')
            elif (token == 'cap') and ('lang' not in i):
                list_label.append('uom')
            elif (token == 'kopi') or (token == 'susu'):
                list_label.append('flavor')
            elif ('dua belas' in i) and ((token == 'dua') or (token == 'belas')):
                list_label.append('desc')
            elif ('enam belas' in i) and ((token == 'enam') or (token == 'belas')):
                list_label.append('desc')
            elif ('dua puluh' in i) and ((token == 'dua') or (token == 'puluh')):
                list_label.append('desc')
            elif (('tujuh enam' in i)or('tujuh puluh enam' in i)) and ((token =='tujuh')or(token=='puluh')or(token=='enam')):
                list_label.append('desc')
            elif (token == 'bos') and ('nutri' not in i):
                list_label.append('eoo')
            elif (token == 'dua') and ('mx' not in i):
                list_label.append('numeric')
            elif (token == 'tiga') and ('kaki' not in i):
                list_label.append('numeric')
            elif (token == 'wan') and ('yusi' not in i):
                list_label.append('numeric')
            elif (token == 'tujuh') and ('delapan' not in i):
                list_label.append('numeric')
            elif ((token == 'ekstra') or (token == 'ektra')) and (tokenize_word.index(token) == 1):
                list_label.append('desc')
            else :
                idx_vocab = list_vocab.index(token.lower())
                list_label.append(list_vocab_label[idx_vocab].split(':')[1])
        else :
            out_of_vocab.append(token)
    result.append(i+':'+(' '.join(list_label)))

out_of_vocab = set(out_of_vocab)

In [7]:
out_of_vocab

set()

In [8]:
path_train_data = '../data/clean/data-with-label.txt'
path_test_data = '../data/testing/clear-sent-comb-with-label.txt'

with open(path_test_data,'w') as outfile:
    for line in tqdm(result):
        outfile.write('%s\n' % line)

In [9]:
result[-10:]

['enam kotak garpit:numeric uom brand',
 'enam kotak gepe:numeric uom brand',
 'tujuh kotak garpit:numeric uom brand',
 'tujuh kotak gepe:numeric uom brand',
 'delapan kotak garpit:numeric uom brand',
 'delapan kotak gepe:numeric uom brand',
 'sembilan kotak garpit:numeric uom brand',
 'sembilan kotak gepe:numeric uom brand',
 'sepuluh kotak garpit:numeric uom brand',
 'sepuluh kotak gepe:numeric uom brand']

In [10]:
len(result)

127466